In [100]:
import pandas as pd
import ast

In [101]:
df_credits = pd.read_csv('../data_original/credits.csv',sep=',')
df_movies = pd.read_csv('../data_original/movies_dataset.csv',sep=',',low_memory=False)

**************************************** TRANSNFORMACIONES ****************************************

6. ELIMINAR LAS COLUMAN QUE NO SERAN UTILIZADAS, video, imdb_id, adult, original_title, poster_path Y homepage

In [105]:
df_movies2 = df_movies.drop(['video','imdb_id','adult','original_title','poster_path','homepage'],axis=1) #Crear nuevo dataframe para poder acceder al inicial de ser necesario\

1. DESANIDAR CAMPOS 

In [106]:
def desanidarDiccionario(df:pd.DataFrame, columna:str):
    '''Devuelve en forma de lista cada el valor de todos los "name" presentes en el diccionario'''
    df = df[columna]
    listName = []

    for i in df:
        try:
            conver = ast.literal_eval(i)
            listName.append(conver["name"])
        except Exception:
            listName.append('') #Si no pertenece a ninguna coleccion se deja es blanco

    newdict = {
        "name_collection": listName,
        }
    df_return = pd.DataFrame(data=newdict) #
    return df_return

df_movies2['belongs_to_collection'] = desanidarDiccionario(df_movies2, 'belongs_to_collection')

In [107]:
def desanidarLista(df:pd.DataFrame, columna:str):
    '''Desanida las columas que tiene una lista de diccionarios y devuelve los valores de la clave "name" de cada diccionario, el retur se hace en forma de lista'''
    df = df[columna]
    listName = [] #Se inicializa la lista en la que va almacenada el/los dato para el nuevo df

    for i in df:
        listTemporal = [] #Lista para almacenar el/los valores de los diccionarios con la clave "name"
        try:
            conver = ast.literal_eval(i)
            # print(type(conver),conver, len(conver)) ## hasta aqui ya se que es una lista y tiene 3 diccionarios dentro
            for item in range(len(conver)):
                listTemporal.append(conver[item]['name']) #Para cada diccionario presente en la lista se almacena el valor de "name"
        except Exception as e:
            listName.append('') #Si el formato no es str se deja en blanco
            continue
        listName.append(listTemporal)

    newdata = {
        'name': listName
    }

    df_return = pd.DataFrame(data=newdata)
    return df_return

df_movies2['genres'] = desanidarLista(df_movies2,'genres')
df_movies2['production_companies'] = desanidarLista(df_movies2,'production_companies')
df_movies2['production_countries'] = desanidarLista(df_movies2,'production_countries')

2. LOS VALORES NULOS DE LOS CAMPOS revenue, budget DEBEN SER RELLENADOS POR EL NUMERO 0

In [108]:
def eliminarNumberNulos(data:pd.DataFrame,columna, tipoDato=0, reemplazo='0'):
    '''Devuelve una columna sin nulos o NaN y reemplaza esos vales con el parametro "reemplazo"'''
    col = data[columna]
    try:
        col = pd.to_numeric(col, errors='coerce')
        col = col.fillna(reemplazo)
        if tipoDato != 0:
            col = col.astype(tipoDato)
    except Exception as e:
        pass
    return col

df_movies2['budget'] = eliminarNumberNulos(df_movies2,'budget',int)
df_movies2['revenue'] = eliminarNumberNulos(df_movies2,'revenue',int)

3. LOS VALORES NULOS DEL CAMPO release_date DEBEN ELIMINARSE

In [109]:
df_movies2 = df_movies2.dropna(subset=['release_date']) #Al ser una columna de info relevante se eliminan las filas con nulos en la columna
for i in df_movies2['release_date']: #Si no obtenemos ninguna salida quiere decir que los valores dela columna se convirtieron correctamente
    if not isinstance(i,str):
        print(i,type(i))

4. DE HABER FECHAS, DEBERAN TENER EL FORMATO "AAA-mm-dd", ADEMAS DEBERAN CREAR LA COLUMNA release_year DONDE EXTRAERAN EL AÑO DE LA FECHA DE ESTRENO

In [110]:
df_movies2['release_date'] = pd.to_datetime(df_movies2['release_date'], errors='coerce') #Se convierten los valores de tipo str a timestamp, los errores de cambian por nan
indiciesNa = df_movies2[df_movies2['release_date'].isna()].index #Se guardan los indices de los valores nulos
df_movies2 = df_movies2.drop(indiciesNa) #Se eliminan las filas que tienen nulos en la columna "release_date"
df_movies2['release_year'] = df_movies2['release_date'].dt.year.astype(int) #Se extrae el year en tipo int para crea la nueva columna cono los datos extraidos y convertidos

5. CREAR LA COLUMNA CON EL RETORNO DE INVERSION, LLAMADA return CON LOS CAMPOS revenue Y budget, DIVIDIENDO ESTAS DOL ULTIMAS revenue/budget, CUANDO NO HAY DATOS DISPONIBLES PARA CALCULARLO, DEBERAN TOMAR EL VALOR 0

In [111]:
def calcularReturn(col1, col2):
    if col1 > 0 and col2 > 0: #Solo devuelve el calculo si ambos valores son positivos y mayor que 0
        return round(col1 / col2, 4)
    else:
        return 0 #Retorna 0 si alguno de los 2 datos no esta disponible o es menor a 0

df_movies2['return'] = df_movies2.apply(lambda row: calcularReturn(row['revenue'],row['budget']),axis=1)

CONSIDERACIONES FINALES PERSONALES

In [112]:
#Se eliminan las columnas que no se van a usar
df_movies2 = df_movies2.drop(['production_countries','spoken_languages','status','runtime','original_language'],axis=1)

#Se cambia el tipo de dato o uno en el que se sienta mas a gusto manipulando
df_movies2['popularity'] = eliminarNumberNulos(df_movies2,'popularity',float)

#Se verifica que no existan duplicados en columnas relevante
df_movies2['id'] = df_movies2['id'].drop_duplicates()
df_movies2['id'] = eliminarNumberNulos(df_movies2,'id',int)

#Se elimina otra columna que no sera usada en la API o el ML
df_movies2 = df_movies2.drop(['tagline'], axis=1)

#Filtramos solo las peliculas lanzadas despues del anio 1980
# df_movies2 = df_movies2[df_movies2['release_year']>1980]

#para evitar algun tipo de problema con el ML eliminamos los nulos 
df_movies2 = df_movies2.dropna()

#Si se diera el caso, se resetean los indices
df_movies2 = df_movies2.reset_index(drop=True)

In [113]:
df_movies2.to_parquet('../data/dataset_movies.parquet',index=False)

In [114]:
df_movies2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44435 entries, 0 to 44434
Data columns (total 14 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   belongs_to_collection  44435 non-null  object        
 1   budget                 44435 non-null  int32         
 2   genres                 44435 non-null  object        
 3   id                     44435 non-null  int32         
 4   overview               44435 non-null  object        
 5   popularity             44435 non-null  float64       
 6   production_companies   44435 non-null  object        
 7   release_date           44435 non-null  datetime64[ns]
 8   revenue                44435 non-null  object        
 9   title                  44435 non-null  object        
 10  vote_average           44435 non-null  float64       
 11  vote_count             44435 non-null  float64       
 12  release_year           44435 non-null  int32         
 13  r